# Predicting Effective Arguments
### Proyecto 2 - Análisis Exploratorio

- Daniel Alfredo Rayo Roldan
- Gerardo Gabriel Pineda Riveiro
- Ricardo Chuy
- Irving Fabricio Morales Acosta

[Link al repositorio aquí](https://github.com/chuy-zip/PROYECTO2_DS)


In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords as nltk_stop
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import requests

In [2]:
df = pd.read_csv("./data/train.csv")

## Diccionario de variables

Se trabajó con un dataset que comprendía fragmento de ensayos argumentativos escritos por niños de los Estados Unidos entre 6to y 12vo grado; el conjunto de datos estaba compuesto por 36765 muestras y 5 variables, de las cuales se explicaran a continuación:

|Nombre         |Tipo |Significado      | Posibles valores |
|------         |--------               |------     |----- |
|discourse_id   | categórica | ID para el discurso | -- |
|essay_id   | categórica | ID del ensayo al que pertenece el discurso | -- |
|discourse_text   | categórica | Texto del discurso | -- |
|discourse_type   | categórica | Type de discurso | Lead, Position, Claim, Counterclaim, Rebuttal, Evidence, Concluding Statement|
|discourse_effectiveness   | categórica| Efectividad del discurso | Ineffective, Adequate, Effective |

### Sobre los diferentes tipo de discursos

El dataset fue creado a partir de un corpuse de 4191 ensayos escritos por niños. Estos ensayos fueron analizados por expertos que identificaron diferentes secciones de cada ensayo en "discursos" o "pasajes", estos los diferentes tipos de pasajes:

1. **Introducción (Lead):** Fragmento de inicio que comienza con una estadística, una cita, una descripción o algún otro recurso para captar la atención del lector y señalar la tesis.

2. **Postura (Position):** Una opinión o conclusión sobre la pregunta principal.

3. **Argumento (Claim):** Una afirmación que respalda la postura.

4. **Contraargumento (CounterClaim):** Una afirmación que refuta otro argumento o ofrece una razón contraria a la postura.

5. **Refutación (Rebuttal):** Un argumento que refuta una contraargumento.

6. **Evidencia (Evidence):**  Ideas o ejemplos que respaldan los argumentos, contraargumentos o refutaciones.

7. **Conclusión (Concluding Statement):**  una declaración final que reafirma los argumentos y postura.

Un ensayo puede estar compuesto por varios fragmentos de diferentes tipos ( varios fragmentos comparten el mismo `essay_id`), cada parte a su vez podía caer en una de tres calificaciones: 

1. **Efectiva**
2. **Adecuada**
3. **Inadecuada**

In [3]:
df.head(10)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
5,36a565e45db7,007ACE74B050,"though some say that life on Mars does exist, ...",Rebuttal,Ineffective
6,fb65fe816ba3,007ACE74B050,"It says in paragraph 7, on April 5, 1998, Mars...",Evidence,Adequate
7,4e472e2584fa,007ACE74B050,Everyone who thought it was made by alieans ev...,Counterclaim,Adequate
8,28a94d3ee425,007ACE74B050,Though people were not satified about how the ...,Concluding Statement,Adequate
9,d226f06362f5,00944C693682,Limiting the usage of cars has personal and pr...,Lead,Effective


## Preprocesamiento

Para poder analizar más a detalle las carácteristicas de los diferentes tipos primero se aplicará un preprocesamiento a los fragmentos de texto para poder remover las palabras conectivas, signos de puntuación y minimizar el texto. 

Sin embargo, primero hay que comprobar por la presencia de valores nulos o faltantes:

In [4]:
df.isnull().sum()

discourse_id               0
essay_id                   0
discourse_text             0
discourse_type             0
discourse_effectiveness    0
dtype: int64

Habiendo comprobado que no hay datos nulos evidentes se procederá a agregar procesar el texto aplicando las siguientes transformaciones: 

- Todas las letras a minúscula.
- Eliminación de signos de puntuación.
- Eliminiación de palabras conectivas.
- Eliminación de caracteres de escapado.
- Traducción de verbos a su versión en infinitivo
- Traducción de sustantivos a su versión en singular


In [5]:
resources = ["punkt", "punkt_tab", "stopwords", "wordnet"]
for r in resources:
    nltk.download(r, quiet=True)

nltk.data.path.append('/home/smaug/nltk_data')

In [6]:
wnl = WordNetLemmatizer()
stopwords = set(nltk_stop.words("english"))
negs = {"no", "not", "n't"}
stopwords = {w for w in stopwords if w.lower() not in negs}

def preprocess_essay(text):
    text = text.lower()
    
    text = ' '.join(text.split())
    
    tokens = word_tokenize(text)
    
    processed_tokens = []
    for token in tokens:
        if (len(token) > 1 and 
            token.isalpha() and 
            token not in stopwords):
            lemmatized = wnl.lemmatize(token)
            processed_tokens.append(lemmatized)
    
    return processed_tokens

In [7]:
df["tokens"] = df["discourse_text"].map(preprocess_essay)
df["text_clean"] = df["tokens"].map(lambda ts: " ".join(ts))

Este es un ejemplo de las transformaciones que sufrio el texto, donde se terminó al finar con una conjunto de texto de palabras normalizadas:

In [8]:
print(df[["discourse_text","tokens","text_clean"]].head(1))

                                      discourse_text  \
0  Hi, i'm Isaac, i'm going to be writing about h...   

                                              tokens  \
0  [hi, isaac, going, writing, face, mar, natural...   

                                          text_clean  
0  hi isaac going writing face mar natural landfo...  


Estas serían las primeras lineas del dataframe trabajado, el cual se guardo en su propio csv, para evitar procesar todo nuevamente.

In [9]:
df.head(10)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,tokens,text_clean
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,"[hi, isaac, going, writing, face, mar, natural...",hi isaac going writing face mar natural landfo...
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,"[perspective, think, face, natural, landform, ...",perspective think face natural landform dont t...
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"[think, face, natural, landform, no, life, mar...",think face natural landform no life mar descov...
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,"[life, mar, would, know, reason, think, natura...",life mar would know reason think natural landf...
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,"[people, thought, face, formed, alieans, thoug...",people thought face formed alieans thought lif...
5,36a565e45db7,007ACE74B050,"though some say that life on Mars does exist, ...",Rebuttal,Ineffective,"[though, say, life, mar, exist, think, no, lif...",though say life mar exist think no life mar
6,fb65fe816ba3,007ACE74B050,"It says in paragraph 7, on April 5, 1998, Mars...",Evidence,Adequate,"[say, paragraph, april, mar, global, surveyor,...",say paragraph april mar global surveyor flew c...
7,4e472e2584fa,007ACE74B050,Everyone who thought it was made by alieans ev...,Counterclaim,Adequate,"[everyone, thought, made, alieans, even, thoug...",everyone thought made alieans even though not ...
8,28a94d3ee425,007ACE74B050,Though people were not satified about how the ...,Concluding Statement,Adequate,"[though, people, not, satified, landform, natu...",though people not satified landform natural la...
9,d226f06362f5,00944C693682,Limiting the usage of cars has personal and pr...,Lead,Effective,"[limiting, usage, car, personal, professional,...",limiting usage car personal professional suppo...


In [10]:
df_clean = df[["discourse_id", "essay_id", "discourse_type", "discourse_effectiveness", "text_clean"]]
df_clean.to_csv("./data/train_clean.csv")

## Sugerencias Chuy

- Hay diferentes tipos de pasajas así que vale la pena investigar la frecuencia de cada pasaje.
- Analizar la "Adecuado, ineffectivo, e inadecuado", en cada categoria del punto anterior
- Analizar los digramas y trigramas de cada categoría.
- Calcular longitud promedio y varianza de los pasajes.